In [54]:
# importing the preprocessed dataset

import json

jsonFile=json.load(open("preprocessed_dataset.json"))
class_label=["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]

clean_text= jsonFile["clean_text"]
result= jsonFile["result"]

for i in clean_text:
    i+= [-1]* (2041- len(i))

In [54]:
# mapreduce-py installation (for using mapreduce within python)

!pip install mapreduce-py

In [50]:
# mapping text to numerical values

# def map_text():
x_dict={}
y_dict={}
hash_val={}

x_val=0
y_val=0

X = []
Y = []

# mapping and transforming text to numerical values
for i in range(len(clean_text)):
    store=[]
    for j in clean_text[i]:
        if j in x_dict:
            store.append(x_dict[j])
        else:
            x_dict[j]=x_val
            x_val+=1
            store.append(x_dict[j])
    X.append(store)

    s=''.join([str(x) for x in result[i]])
    if s in y_dict:
        Y.append(y_dict[s])
    else:
        y_dict[s]=y_val
        hash_val[y_val]=s
        y_val+=1
        Y.append(y_dict[s])
#     return X, Y

In [29]:
from sklearn import tree

def train(X,Y):
    arr=[]
    for i in X:
       i+= [-1]* (2041- len(i))

    from sklearn import tree
    clf = tree.DecisionTreeClassifier()
    clf = clf.fit(X, Y)
    return clf


In [38]:
def test(X, clf):
    x=clf.predict(X)
    test_set= X[0:int((0.2)*len(X))]
    x=clf.predict(X)

    labels = result[0:int((0.2)*len(X))]
    predictions = x
    from sklearn.metrics import precision_score

#     p=precision_score(labels, x, average='micro')
#     r=recall_score(labels, x, average='micro')
#     a=accuracy_score(labels, x, average='micro')
    
    return predictions
    

In [121]:
# training and testing decision tree

from mapreduce import *

MR=MapReducer().prefilter(map_text).mapper(train).reducer(test, None)
res = MR(range(len(MR)))
print('MR Result  :', res)

accuracy, precision, recall=res
print("accuracy:", accuracy)
print("precision:", precision)
print("recall:", recall)

print("F1 Score:", ((2*precision*recall)/(precision+recall)))

MR Result  : (0.9993836671802774, 0.9893898305084745, 0.9793959938366719)
accuracy: 0.9993836671802774
precision: 0.9893898305084745
recall: 0.9793959938366719
F1 Score: 0.9843675471048782


In [ ]:
# svm 
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
def train():
    from sklearn.svm import SVC
    clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
    clf.fit(X, Y)
    return clf


In [7]:
# training and testing svm

from mapreduce import *

MR=MapReducer().prefilter(map_text).mapper(train).reducer(test, None)
res = MR(range(len(MR)))

accuracy, precision, recall=res
print("accuracy:", accuracy)
print("precision:", precision)
print("recall:", recall)

print("F1 Score:", ((2*precision*recall)/(precision+recall)))

accuracy: 94.94144838212635
precision: 86.39671802773499
recall: 93.04261941448384
F1 Score: 89.59659647320666


In [39]:
# prediction with sample reddit data

# fetching reddit data
import http.client

conn = http.client.HTTPSConnection("reddit3.p.rapidapi.com")

headers = {
    'x-rapidapi-host': "reddit3.p.rapidapi.com",
    'x-rapidapi-key': "52e70ee3a0mshe32a5a1bbaa67bcp1e2257jsndfce50b84406"
    }

conn.request("GET", "/all?url=https%3A%2F%2Fwww.reddit.com%2Fr%all", headers=headers)

res = conn.getresponse()
data = res.read()

text_data=data["replies"]
text_data=text_data[0:min(20, len(text_data))]


# predicting

res= test(text_data, train(X,Y))
print(res)

[ 4  3  1  3  4  1  1  5  3  1  3  1  1 10  1  3  1  3  0  1  1]


In [57]:
# printing hash values of the results

for i in res:
    print(hash_val[i])
    r=""
    for j in hash_val[i]:
        r+=class_label[int(j)]
        r+=' '
    print(r)
        

02
toxic obscene 
024
toxic obscene insult 
0
toxic 
024
toxic obscene insult 
02
toxic obscene 
0
toxic 
0
toxic 
03
toxic threat 
024
toxic obscene insult 
0
toxic 
024
toxic obscene insult 
0
toxic 
0
toxic 
012
toxic severe_toxic obscene 
0
toxic 
024
toxic obscene insult 
0
toxic 
024
toxic obscene insult 
0124
toxic severe_toxic obscene insult 
0
toxic 
0
toxic 
